In [1]:
import tensorflow as tf
# tf.enable_eager_execution()
import numpy as np
import datetime
import os
import argparse
import matplotlib.pyplot as plt
import pandas as pd
import json
import glob
tf.__version__

'1.15.0'

In [2]:
# Run this when train with all data
data_info = {
   "train_unlabel": 0, 
    "train_label": 0, 
    "validation": 0, 
    "test": 0
}
for f in glob.glob('./Data/*/datainfo.txt'):
    data_read = json.load(open(f))
    for key in data_info.keys():
        data_info[key] += data_read[key]
        
data_info

{'train_unlabel': 8166200,
 'train_label': 907358,
 'validation': 2268392,
 'test': 4970811}

## Idea to improve:

1) Tune n_l1, n_l2 and z_dim
2) Generator, Regualrization and Semi phase with different learning rate
3) Discriminator wih smaller learning rate
4) Consider about regularize autoencoder: parse penalty or variational 

# Define Model

In [3]:
# Parameters
# Put the types of attack here
attack = 'all' # DoS, Fuzzy, gear, RPM, all
results_path = './Results/{}/'.format(attack)
data_path = './Data/*/'
# data_path = './Data/{}/'.format(attack)
# data_info = json.load(open(data_path + 'datainfo.txt'))
print('Data info: ', data_info)
input_dim = 29 * 29
n_l1 = 1000
n_l2 = 1000
z_dim = 10
batch_size = 100
n_epochs = 1000
learning_rate = 0.001
beta1 = 0.9
n_labels = 2
n_labeled = data_info['train_label']
validation_size = data_info['validation']

Data info:  {'train_unlabel': 8166200, 'train_label': 907358, 'validation': 2268392, 'test': 4970811}


In [4]:
def dense(x, n1, n2, name):
    """
    Used to create a dense layer.
    :param x: input tensor to the dense layer
    :param n1: no. of input neurons
    :param n2: no. of output neurons
    :param name: name of the entire dense layer.i.e, variable scope name.
    :return: tensor with shape [batch_size, n2]
    """
    with tf.variable_scope(name, reuse=None):
        weights = tf.get_variable("weights", shape=[n1, n2],
                                  initializer=tf.random_normal_initializer(mean=0., stddev=0.01))
        bias = tf.get_variable("bias", shape=[n2], initializer=tf.constant_initializer(0.0))
        out = tf.add(tf.matmul(x, weights), bias, name='matmul')
        return out

In [5]:
def encoder(x, reuse=False, supervised=False):
    """
    Encode part of the autoencoder.
    :param x: input to the autoencoder
    :param reuse: True -> Reuse the encoder variables, False -> Create or search of variables before creating
    :param supervised: True -> returns output without passing it through softmax,
                       False -> returns output after passing it through softmax.
    :return: tensor which is the classification output and a hidden latent variable of the autoencoder.
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Encoder'):
        e_dense_1 = tf.nn.relu(dense(x, input_dim, n_l1, 'e_dense_1'))
        e_dense_2 = tf.nn.relu(dense(e_dense_1, n_l1, n_l2, 'e_dense_2'))
        latent_variable = dense(e_dense_2, n_l2, z_dim, 'e_latent_variable')
        cat_op = dense(e_dense_2, n_l2, n_labels, 'e_label')
        if not supervised:
            softmax_label = tf.nn.softmax(logits=cat_op, name='e_softmax_label')
        else:
            softmax_label = cat_op
        return softmax_label, latent_variable

In [6]:
def decoder(x, reuse=False):
    """
    Decoder part of the autoencoder.
    :param x: input to the decoder
    :param reuse: True -> Reuse the decoder variables, False -> Create or search of variables before creating
    :return: tensor which should ideally be the input given to the encoder.
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Decoder'):
        d_dense_1 = tf.nn.relu(dense(x, z_dim + n_labels, n_l2, 'd_dense_1'))
        d_dense_2 = tf.nn.relu(dense(d_dense_1, n_l2, n_l1, 'd_dense_2'))
        output = tf.nn.sigmoid(dense(d_dense_2, n_l1, input_dim, 'd_output'))
        return output

In [7]:
def discriminator_gauss(x, reuse=False):
    """
    Discriminator that is used to match the posterior distribution with a given gaussian distribution.
    :param x: tensor of shape [batch_size, z_dim]
    :param reuse: True -> Reuse the discriminator variables,
                  False -> Create or search of variables before creating
    :return: tensor of shape [batch_size, 1]
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Discriminator_Gauss'):
        dc_den1 = tf.nn.relu(dense(x, z_dim, n_l1, name='dc_g_den1'))
        dc_den2 = tf.nn.relu(dense(dc_den1, n_l1, n_l2, name='dc_g_den2'))
        output = dense(dc_den2, n_l2, 1, name='dc_g_output')
        return output

In [8]:
def discriminator_categorical(x, reuse=False):
    """
    Discriminator that is used to match the posterior distribution with a given categorical distribution.
    :param x: tensor of shape [batch_size, n_labels]
    :param reuse: True -> Reuse the discriminator variables,
                  False -> Create or search of variables before creating
    :return: tensor of shape [batch_size, 1]
    """
    if reuse:
        tf.get_variable_scope().reuse_variables()
    with tf.name_scope('Discriminator_Categorial'):
        dc_den1 = tf.nn.relu(dense(x, n_labels, n_l1, name='dc_c_den1'))
        dc_den2 = tf.nn.relu(dense(dc_den1, n_l1, n_l2, name='dc_c_den2'))
        output = dense(dc_den2, n_l2, 1, name='dc_c_output')
        return output

# Define data

In [9]:
def read_tfrecord(example):
    feature_description = {
    'input_features': tf.io.FixedLenFeature([input_dim], tf.int64),
    'label': tf.io.FixedLenFeature([1], tf.int64)
    }
    return tf.io.parse_single_example(example, feature_description)

In [10]:
def data_from_tfrecord(tf_filepath, batch_size, repeat_time):
    data = tf.data.TFRecordDataset(tf_filepath)
    data = data.map(read_tfrecord)
    data = data.shuffle(1)
    data = data.repeat(repeat_time + 1)
    data = data.batch(batch_size)
    # print(tf.data.experimental.cardinality(data))
    iterator = data.make_one_shot_iterator()
    return iterator.get_next()

In [11]:
# For all
train_unlabel = data_from_tfrecord(glob.glob(data_path + 'train_unlabel'), batch_size, n_epochs)
train_label = data_from_tfrecord(glob.glob(data_path + 'train_label'), batch_size, n_epochs)
validation = data_from_tfrecord(glob.glob(data_path + 'val'), batch_size, n_epochs)

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [11]:
#For each attack
train_unlabel = data_from_tfrecord(data_path + 'train_unlabel', batch_size, n_epochs)
train_label = data_from_tfrecord(data_path + 'train_label', n_labeled, 1)
validation = data_from_tfrecord(data_path + 'val', batch_size, n_epochs)

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [12]:
def data_helper(data_tf, sess):
    data = sess.run(data_tf)
    x, y = data['input_features'], data['label']
    size = x.shape[0]
    y_one_hot = np.eye(n_labels)[y].reshape([size, n_labels])
    return x, y_one_hot

In [13]:
# init = tf.global_variables_initializer()
# with tf.Session() as sess:
#     sess.run(init)
#     x_l, y_l = data_helper(train, sess)
#     print(x_l.shape, y_l.shape)

In [13]:
# Placeholders for input data and the targets
x_input = tf.placeholder(dtype=tf.float32, shape=[batch_size, input_dim], name='Input')
x_input_l = tf.placeholder(dtype=tf.float32, shape=[batch_size, input_dim], name='Labeled_Input')
y_input = tf.placeholder(dtype=tf.float32, shape=[batch_size, n_labels], name='Labels')
x_target = tf.placeholder(dtype=tf.float32, shape=[batch_size, input_dim], name='Target')
real_distribution = tf.placeholder(dtype=tf.float32, shape=[batch_size, z_dim], name='Real_distribution')
categorial_distribution = tf.placeholder(dtype=tf.float32, shape=[batch_size, n_labels],
                                         name='Categorical_distribution')
manual_decoder_input = tf.placeholder(dtype=tf.float32, shape=[1, z_dim + n_labels], name='Decoder_input')

# Loss

In [14]:
# Reconstruction Phase
# Encoder try to predict both label and latent space of the input, which will be feed into Decoder to reconstruct the input
# The process is optimized by autoencoder_loss which is the MSE of the decoder_output and the orginal input
with (tf.variable_scope(tf.get_variable_scope())):
    encoder_output_label, encoder_output_latent = encoder(x_input)
    decoder_input = tf.concat([encoder_output_label, encoder_output_latent], 1)
    decoder_output = decoder(decoder_input)

autoencoder_loss = tf.reduce_mean(tf.square(x_target - decoder_output))
autoencoder_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1).minimize(autoencoder_loss)

In [15]:
# Regularization Phase
# Train both 2 discriminator of gaussian and categorical to detect the output from encoder
with (tf.variable_scope(tf.get_variable_scope())):
    # Discriminator for gaussian
    d_g_real = discriminator_gauss(real_distribution)
    d_g_fake = discriminator_gauss(encoder_output_latent, reuse=True)
# Need to seperate dicriminator of gaussian and categorical
with (tf.variable_scope(tf.get_variable_scope())):
    # Discrimnator for categorical
    d_c_real = discriminator_categorical(categorial_distribution)
    d_c_fake = discriminator_categorical(encoder_output_label, reuse=True)

# Discriminator gaussian loss 
dc_g_loss_real = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(d_g_real), logits=d_g_real))
dc_g_loss_fake = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(d_g_fake), logits=d_g_fake))
dc_g_loss = dc_g_loss_real + dc_g_loss_fake

# Discriminator categorical loss
dc_c_loss_real = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(d_c_real), logits=d_c_real))
dc_c_loss_fake = tf.reduce_mean(
                    tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.zeros_like(d_c_fake), logits=d_c_fake))
dc_c_loss = dc_c_loss_fake + dc_c_loss_real

all_variables = tf.trainable_variables()
dc_g_var = [var for var in all_variables if 'dc_g_' in var.name]
dc_c_var = [var for var in all_variables if 'dc_c_' in var.name]
discriminator_g_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                                       beta1=beta1).minimize(dc_g_loss, var_list=dc_g_var)
discriminator_c_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,
                                                       beta1=beta1).minimize(dc_c_loss, var_list=dc_c_var)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
# Generator loss
generator_g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(d_g_fake), logits=d_g_fake))
generator_c_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(d_c_fake), logits=d_c_fake))
generator_loss = generator_g_loss + generator_c_loss

en_var = [var for var in all_variables if 'e_' in var.name]
generator_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1).minimize(generator_loss, var_list=en_var)

In [17]:
# Semi-Supervised Classification Phase
# Train encoder with a small amount of label samples
with tf.variable_scope(tf.get_variable_scope()):
    encoder_output_label_, _ = encoder(x_input_l, reuse=True, supervised=True)
    
# Classification accuracy of encoder
correct_pred = tf.equal(tf.argmax(encoder_output_label_, 1), tf.argmax(y_input, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

supervised_encoder_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_input, logits=encoder_output_label_))
supervised_encoder_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1).minimize(supervised_encoder_loss, var_list=en_var)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [18]:
def form_results():
    """
    Forms folders for each run to store the tensorboard files, saved models and the log files.
    :return: three string pointing to tensorboard, saved models and log paths respectively.
    """
    folder_name = "/{0}_{1}_{2}_{3}_{4}_{5}_Semi_Supervised". \
        format(datetime.datetime.now(), z_dim, learning_rate, batch_size, n_epochs, beta1)
    tensorboard_path = results_path + folder_name + '/Tensorboard'
    saved_model_path = results_path + folder_name + '/Saved_models/'
    log_path = results_path + folder_name + '/log'
    if not os.path.exists(results_path + folder_name):
        os.mkdir(results_path + folder_name)
        os.mkdir(tensorboard_path)
        os.mkdir(saved_model_path)
        os.mkdir(log_path)
    return tensorboard_path, saved_model_path, log_path

In [19]:
def next_batch(x, y, batch_size):
    """
    Used to return a random batch from the given inputs.
    :param x: Input images of shape [None, 784]
    :param y: Input labels of shape [None, 10]
    :param batch_size: integer, batch size of images and labels to return
    :return: x -> [batch_size, 784], y-> [batch_size, 10]
    """
    index = np.arange(n_labeled)
    random_index = np.random.permutation(index)[:batch_size]
    return x[random_index], y[random_index]

# Training

In [ ]:
init = tf.global_variables_initializer()

# Tensorboard visualization
tf.summary.scalar(name='Autoencoder Loss', tensor=autoencoder_loss)
tf.summary.scalar(name='Discriminator gauss Loss', tensor=dc_g_loss)
tf.summary.scalar(name='Discriminator categorical Loss', tensor=dc_c_loss)
tf.summary.scalar(name='Generator Loss', tensor=generator_loss)
tf.summary.scalar(name='Supervised Encoder Loss', tensor=supervised_encoder_loss)
# tf.summary.scalar(name='Supervised Encoder Accuracy', tensor=accuracy)
tf.summary.histogram(name='Encoder Gauss Distribution', values=encoder_output_latent)
tf.summary.histogram(name='Real Gauss Distribution', values=real_distribution)
tf.summary.histogram(name='Encoder Categorical Distribution', values=encoder_output_label)
tf.summary.histogram(name='Real Categorical Distribution', values=categorial_distribution)

summary_op = tf.summary.merge_all()

accuracies = []
# Saving the model
saver = tf.train.Saver()
step = 0
# Early stopping
save_sess = None
best_acc = 1
stop = False
last_improvement = 0
require_improvement = 20

with tf.Session() as sess:
    if True:
        tensorboard_path, saved_model_path, log_path = form_results()
        sess.run(init)
        writer = tf.summary.FileWriter(logdir=tensorboard_path, graph=sess.graph)
        # x_l, y_l = mnist.test.next_batch(n_labeled)
        #x_l, y_l = data_helper(train_label, sess)
        for epoch in range(n_epochs):
            # Eearly stopping
            if stop:
                print('Eearly stopping')
                break
            n_batches = int(n_labeled / batch_size)
            print("------------------Epoch {}/{}------------------".format(epoch, n_epochs))
            for b in range(1, n_batches + 1):
                z_real_dist = np.random.randn(batch_size, z_dim) * 5.
                real_cat_dist = np.random.randint(low=0, high=2, size=batch_size)
                real_cat_dist = np.eye(n_labels)[real_cat_dist]
                # batch_x_ul, _ = mnist.train.next_batch(batch_size)
                batch_x_ul, _ = data_helper(train_unlabel, sess)
                batch_x_l, batch_y_l = data_helper(train_label, sess)
                
                sess.run(autoencoder_optimizer, feed_dict={x_input: batch_x_ul, x_target: batch_x_ul})
                sess.run(discriminator_g_optimizer,
                         feed_dict={x_input: batch_x_ul, x_target: batch_x_ul, real_distribution: z_real_dist})
                sess.run(discriminator_c_optimizer,
                         feed_dict={x_input: batch_x_ul, x_target: batch_x_ul,
                                    categorial_distribution: real_cat_dist})
                sess.run(generator_optimizer, feed_dict={x_input: batch_x_ul, x_target: batch_x_ul})
                
                sess.run(supervised_encoder_optimizer, feed_dict={x_input_l: batch_x_l, y_input: batch_y_l})
                if b % 100 == 0:
                    a_loss, d_g_loss, d_c_loss, g_loss, s_loss, summary = sess.run(
                        [autoencoder_loss, dc_g_loss, dc_c_loss, generator_loss, supervised_encoder_loss,
                         summary_op],
                        feed_dict={x_input: batch_x_ul, x_target: batch_x_ul,
                                   real_distribution: z_real_dist, y_input: batch_y_l, x_input_l: batch_x_l,
                                   categorial_distribution: real_cat_dist})
                    writer.add_summary(summary, global_step=step)
#                     print("Epoch: {}, iteration: {}".format(i, b))
#                     print("Autoencoder Loss: {}".format(a_loss))
#                     print("Discriminator Gauss Loss: {}".format(d_g_loss))
#                     print("Discriminator Categorical Loss: {}".format(d_c_loss))
#                     print("Generator Loss: {}".format(g_loss))
#                     print("Supervised Loss: {}\n".format(s_loss))
                    with open(log_path + '/log.txt', 'a') as log:
                        log.write("Epoch: {}, iteration: {}\n".format(epoch, b))
                        log.write("Autoencoder Loss: {}\n".format(a_loss))
                        log.write("Discriminator Gauss Loss: {}".format(d_g_loss))
                        log.write("Discriminator Categorical Loss: {}".format(d_c_loss))
                        log.write("Generator Loss: {}\n".format(g_loss))
                        log.write("Supervised Loss: {}".format(s_loss))
                step += 1
                
            if (epoch + 1) % 5 == 0:
                # Calculate the accuracy on validation set
                acc = 0
                # num_batches = int(mnist.validation.num_examples/batch_size)
                num_batches = int(validation_size/batch_size)
                for j in range(num_batches):
                    # Classify unseen validation data instead of test data or train data
                    # batch_x_l, batch_y_l = mnist.validation.next_batch(batch_size=batch_size)
                    batch_x_l, batch_y_l = data_helper(validation, sess)
                    encoder_acc = sess.run(accuracy, feed_dict={x_input_l: batch_x_l, y_input: batch_y_l})
                    acc += encoder_acc
                avg_acc = acc/num_batches
                accuracies.append(avg_acc)
                print("Encoder Classification Accuracy: {}".format(avg_acc))
                with open(log_path + '/log.txt', 'a') as log:
                    log.write("Encoder Classification Accuracy: {}".format(avg_acc))

                # Eearly stopping
                if avg_acc < best_acc:
                    save_sess = sess
                    best_acc = avg_acc
                    last_improvement = 0
                else:
                    last_improvement += 1
                if last_improvement > require_improvement:
                    stop = True
                    sess = save_sess
    
            saver.save(sess, save_path=saved_model_path, global_step=step)

INFO:tensorflow:Summary name Autoencoder Loss is illegal; using Autoencoder_Loss instead.
INFO:tensorflow:Summary name Discriminator gauss Loss is illegal; using Discriminator_gauss_Loss instead.
INFO:tensorflow:Summary name Discriminator categorical Loss is illegal; using Discriminator_categorical_Loss instead.
INFO:tensorflow:Summary name Generator Loss is illegal; using Generator_Loss instead.
INFO:tensorflow:Summary name Supervised Encoder Loss is illegal; using Supervised_Encoder_Loss instead.
INFO:tensorflow:Summary name Encoder Gauss Distribution is illegal; using Encoder_Gauss_Distribution instead.
INFO:tensorflow:Summary name Real Gauss Distribution is illegal; using Real_Gauss_Distribution instead.
INFO:tensorflow:Summary name Encoder Categorical Distribution is illegal; using Encoder_Categorical_Distribution instead.
INFO:tensorflow:Summary name Real Categorical Distribution is illegal; using Real_Categorical_Distribution instead.
------------------Epoch 0/1000--------------

# Test

In [25]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
# test_paths = glob.glob(data_path + 'test')
with tf.Session() as sess:
    attack = 'DoS'
    data_path = './Data/{}/'.format(attack)
    results_path = './Results/{}/'.format(attack)
    data_info = json.load(open(data_path + 'datainfo.txt'))
    
    # Get the latest results folder
    all_results = os.listdir(results_path)
    all_results.sort()
    
    #saver.restore(sess, save_path=tf.train.latest_checkpoint(results_path + '/' +
    #                                                         all_results[1] + '/Saved_models/'))
    saver.restore(sess, save_path=tf.train.latest_checkpoint('./Results/all/2021-06-30 10:04:49.877833_100_0.001_128_1000_0.9_Semi_Supervised/Saved_models'))
    
    # produce_test_result()
    test_size = data_info['test']
    #test = data_from_tfrecord(test_paths, test_batch_size, 1)
    test = data_from_tfrecord(data_path + 'test', batch_size, 1)

    with tf.variable_scope(tf.get_variable_scope()):
        output_label, _ = encoder(x_input_l, reuse=True, supervised=True)

    pred_label = tf.argmax(output_label, 1)
    num_batches = int(test_size / batch_size)
    y_true = []
    y_pred = []
    for _ in range(num_batches):
        x_test, y_test = data_helper(test, sess)
        batch_pred = sess.run(pred_label, feed_dict={x_input_l: x_test})
        batch_label = np.argmax(y_test, axis=1)

        y_pred += batch_pred.tolist()
        y_true += batch_label.tolist()

INFO:tensorflow:Restoring parameters from ./Results/all/2021-06-30 10:04:49.877833_100_0.001_128_1000_0.9_Semi_Supervised/Saved_models/-318960


INFO:tensorflow:Restoring parameters from ./Results/all/2021-06-30 10:04:49.877833_100_0.001_128_1000_0.9_Semi_Supervised/Saved_models/-318960


In [26]:
from sklearn.metrics import confusion_matrix

In [27]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
fnr = fn/(tp + fn)
err = (fn + fp) / (tp + tn + fp + fn)
precision = tp/(tp + fp)
recall = 1 - fnr
f1score = (2 * precision * recall) / (precision + recall)
print('False negative rate: ', fnr)
print('Error rate: ', err)
print('Precision: ', precision)
print('Recall: ', recall)
print('F1 score: ', f1score)

False negative rate:  0.006910531650234958
Error rate:  0.008954683680595972
Precision:  0.9770439690938154
Recall:  0.9930894683497651
F1 score:  0.9850013784657752
